In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_excel('r&r data.xlsx')
df

,product,review,rating,profile_name
0,OnePlus,*Read before you buy!!*,5,Nikhil
1,OnePlus,Near to mid range Perfection,5,Amit
2,OnePlus,Great price!,5,aishwarya
3,OnePlus,Beast in OnePlus.,5,vasu a.
4,OnePlus,Changed to Nord from 6t,5,Amazon Customer
...,...,...,...,...
30607,Redmi,Great but not satisfied for me,2,rakesh
30608,Redmi,Not recommend,2,Amazon Customer
30609,Redmi,WORST MARKETING TACTICS OF FLASH SALE OF REDMI...,2,SREEKUMAR V
30610,Redmi,Display retention issue,2,Abhishek M.


In [3]:
df.drop(['product','profile_name'],axis=1,inplace=True)

In [4]:
df

,review,rating
0,*Read before you buy!!*,5
1,Near to mid range Perfection,5
2,Great price!,5
3,Beast in OnePlus.,5
4,Changed to Nord from 6t,5
...,...,...
30607,Great but not satisfied for me,2
30608,Not recommend,2
30609,WORST MARKETING TACTICS OF FLASH SALE OF REDMI...,2
30610,Display retention issue,2


In [5]:
df['rating'].value_counts()

5    13967
4     8537
3     4172
1     2088
2     1848
Name: rating, dtype: int64

In [6]:
import preprocess_kgptalkie as ps
import re

In [7]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [8]:
df['review'] = df['review'].apply(lambda x: get_clean(x)) 

/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:329: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:329: MarkupResemblesLocatorWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [9]:
df.head()

,review,rating
0,read before you buy,5
1,near to mid range perfection,5
2,great price,5
3,beast in oneplus,5
4,changed to nord from 6t,5


In [10]:
df.tail()

,review,rating
30607,great but not satisfied for me,2
30608,not recommend,2
30609,worst marketing tactics of flash sale of redmi...,2
30610,display retention issue,2
30611,camera quality is bad yz,2


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier

In [12]:
tfidf=TfidfVectorizer(max_features=20000,ngram_range=(1,5),analyzer='char')

In [13]:
X=tfidf.fit_transform(df['review'])
y=df['rating']

In [14]:
X.shape,y.shape

((30612, 20000), (30612,))

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:
X_train.shape

(24489, 20000)

In [17]:
clf=LinearSVC(C=20,class_weight='balanced')
clf.fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=20, class_weight='balanced')

In [18]:
y_pred=clf.predict(X_test)

In [19]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.44      0.46      0.45       436
           2       0.23      0.27      0.25       385
           3       0.31      0.28      0.29       831
           4       0.38      0.39      0.38      1721
           5       0.66      0.66      0.66      2750

    accuracy                           0.49      6123
   macro avg       0.41      0.41      0.41      6123
weighted avg       0.49      0.49      0.49      6123



In [20]:
mnb = MultinomialNB().fit(X_train, y_train)

In [21]:
mnb_pred = mnb.predict(X_test)

In [22]:
print(classification_report(y_test,mnb_pred))

              precision    recall  f1-score   support

           1       0.52      0.36      0.43       436
           2       0.20      0.01      0.01       385
           3       0.32      0.39      0.35       831
           4       0.38      0.29      0.33      1721
           5       0.63      0.80      0.70      2750

    accuracy                           0.52      6123
   macro avg       0.41      0.37      0.37      6123
weighted avg       0.48      0.52      0.49      6123



In [23]:
lr = LogisticRegression().fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
lr_pred=lr.predict(X_test)

In [25]:
print(classification_report(y_test,lr_pred))

              precision    recall  f1-score   support

           1       0.54      0.48      0.51       436
           2       0.40      0.08      0.13       385
           3       0.36      0.33      0.35       831
           4       0.40      0.28      0.33      1721
           5       0.62      0.84      0.71      2750

    accuracy                           0.54      6123
   macro avg       0.46      0.40      0.40      6123
weighted avg       0.50      0.54      0.50      6123



In [26]:
knn = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)     

In [27]:
knn_pred = knn.predict(X_test)

In [28]:
print(confusion_matrix(y_test,knn_pred))

[[ 180   53   86   70   47]
 [  85   49  107   86   58]
 [ 105   63  224  274  165]
 [  55   61  199  629  777]
 [  44   23  191  724 1768]]


In [31]:
print(accuracy_score(y_test,knn_pred))


0.46545810877021065


In [32]:
print(classification_report(y_test,knn_pred))

              precision    recall  f1-score   support

           1       0.38      0.41      0.40       436
           2       0.20      0.13      0.15       385
           3       0.28      0.27      0.27       831
           4       0.35      0.37      0.36      1721
           5       0.63      0.64      0.64      2750

    accuracy                           0.47      6123
   macro avg       0.37      0.36      0.36      6123
weighted avg       0.46      0.47      0.46      6123



In [34]:
dtc=DecisionTreeClassifier(criterion='entropy')
dtc.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [35]:
dtc_pred = dtc.predict(X_test)
print(confusion_matrix(y_test,dtc_pred))
print(accuracy_score(y_test,dtc_pred))
print(classification_report(y_test,dtc_pred))

[[ 175   74   79   48   60]
 [  84   84  110   56   51]
 [  89   64  225  240  213]
 [  60   43  203  583  832]
 [  44   31  138  653 1884]]
0.4819532908704883
              precision    recall  f1-score   support

           1       0.39      0.40      0.39       436
           2       0.28      0.22      0.25       385
           3       0.30      0.27      0.28       831
           4       0.37      0.34      0.35      1721
           5       0.62      0.69      0.65      2750

    accuracy                           0.48      6123
   macro avg       0.39      0.38      0.39      6123
weighted avg       0.47      0.48      0.47      6123



In [42]:
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [43]:
rfc_pred = dtc.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))
print(accuracy_score(y_test,rfc_pred))
print(classification_report(y_test,rfc_pred))

[[ 175   74   79   48   60]
 [  84   84  110   56   51]
 [  89   64  225  240  213]
 [  60   43  203  583  832]
 [  44   31  138  653 1884]]
0.4819532908704883
              precision    recall  f1-score   support

           1       0.39      0.40      0.39       436
           2       0.28      0.22      0.25       385
           3       0.30      0.27      0.28       831
           4       0.37      0.34      0.35      1721
           5       0.62      0.69      0.65      2750

    accuracy                           0.48      6123
   macro avg       0.39      0.38      0.39      6123
weighted avg       0.47      0.48      0.47      6123



In [45]:
bnb = BernoulliNB()
bnb.fit(X_train, y_train)     

BernoulliNB()

In [46]:
bnb_pred = bnb.predict(X_test)
print(confusion_matrix(y_test,bnb_pred))
print(accuracy_score(y_test,bnb_pred))
print(classification_report(y_test,bnb_pred))

[[ 199   80   75   31   51]
 [  87   77  127   28   66]
 [ 105  120  254  115  237]
 [  94  106  216  367  938]
 [ 112   64  143  351 2080]]
0.4861995753715499
              precision    recall  f1-score   support

           1       0.33      0.46      0.39       436
           2       0.17      0.20      0.19       385
           3       0.31      0.31      0.31       831
           4       0.41      0.21      0.28      1721
           5       0.62      0.76      0.68      2750

    accuracy                           0.49      6123
   macro avg       0.37      0.39      0.37      6123
weighted avg       0.47      0.49      0.47      6123



In [72]:
x='the product is very good,price worth it'
x=get_clean(x)
vec=tfidf.transform([x])
clf.predict(vec)

array([4])

In [73]:
x='the product is very bad, don,t buy it'
x=get_clean(x)
vec=tfidf.transform([x])
clf.predict(vec)

array([1])

In [75]:
x='charger not included'
x=get_clean(x)
vec=tfidf.transform([x])
clf.predict(vec)

array([3])

In [76]:
x='so good,buy it'
x=get_clean(x)
vec=tfidf.transform([x])
clf.predict(vec)

array([5])

In [49]:
x='the product is very good,price worth it'
x=get_clean(x)
vec=tfidf.transform([x])
lr.predict(vec)

array([5])

In [54]:
import pickle
filename='rating_pred.pkl'
pickle.dump(lr,open(filename,'wb'))

In [56]:
loaded_model=pickle.load(open('rating_pred.pkl','rb'))
result=loaded_model.score(X_test,y_test)
print(result*100)

53.829821982688216


In [57]:
conclusion=pd.DataFrame([loaded_model.predict(X_test)[:],y_pred[:]],index=["Predicted","original"])
conclusion

,0,1,2,3,4,5,6,7,8,9,...,6113,6114,6115,6116,6117,6118,6119,6120,6121,6122
Predicted,5,5,5,5,4,4,5,5,3,3,...,5,5,4,5,3,5,4,5,5,5
original,5,5,5,5,2,4,5,5,3,2,...,5,4,4,5,3,2,4,5,5,4
